In [1]:
import os
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
ax_results_folder = "/home/lucas/Documents/perceptronac/scripts/tradeoffs/ax_results_energy_params_bits/"
prefix = "micro_joules_per_pixel_data_bits_over_data_samples_ax_methods_seed"

stats_dict = dict()

for complexity in [
    "micro_joules_per_pixel",
    "params",
    "model_bits"
]:
    prefix = f"{complexity}_data_bits_over_data_samples_ax_methods_seed"
    for method in ["sobol","ehvi","parego"]:
        
        stats_dict[f"{complexity}_{method}"] = []
        
        set_lens = []
        list_lens = []
        for f in os.listdir(ax_results_folder):
            if f.endswith(".csv") and (prefix in f):
                ax_df = pd.read_csv(os.path.join(ax_results_folder,f))
                method_param_cols = [c for c in ax_df.columns if f"{method}_param_" in c]
                tmp_df = ax_df[method_param_cols].copy()
                          
                list_len = tmp_df.shape[0]
                set_len = tmp_df.drop_duplicates(subset=method_param_cols).shape[0]
                    
                list_lens.append(list_len)
                set_lens.append(set_len)
                
                stats_dict[f"{complexity}_{method}"].append(set_len)
                
        print(complexity,method,min(set_lens),Counter(set_lens),Counter(list_lens))

micro_joules_per_pixel sobol 28 Counter({28: 25}) Counter({28: 25})
micro_joules_per_pixel ehvi 24 Counter({28: 19, 27: 3, 26: 2, 24: 1}) Counter({28: 25})
micro_joules_per_pixel parego 15 Counter({21: 5, 20: 5, 23: 4, 22: 3, 19: 2, 24: 2, 18: 2, 15: 1, 16: 1}) Counter({28: 25})
params sobol 30 Counter({30: 25}) Counter({30: 25})
params ehvi 30 Counter({30: 25}) Counter({30: 25})
params parego 18 Counter({23: 7, 20: 5, 21: 4, 24: 3, 22: 3, 26: 1, 18: 1, 19: 1}) Counter({30: 25})
model_bits sobol 43 Counter({43: 25}) Counter({43: 25})
model_bits ehvi 34 Counter({43: 12, 42: 4, 38: 3, 35: 2, 41: 1, 37: 1, 34: 1, 40: 1}) Counter({43: 25})
model_bits parego 21 Counter({28: 5, 23: 4, 25: 3, 21: 3, 30: 2, 22: 2, 26: 2, 29: 1, 27: 1, 31: 1, 32: 1}) Counter({43: 25})


In [3]:
dfs_dict = dict()

for x_axis in ["micro_joules_per_pixel","model_bits","params"]:
    
    fldr = "/home/lucas/Documents/perceptronac/scripts/tradeoffs/"
    fil3 = f"{x_axis}_data_bits_over_data_samples_ax_methods_avgs_adjusted.csv"
    pth = fldr + fil3
    
    df = pd.read_csv(pth,index_col=0)
    
    df[["sobol_count","ehvi_count","parego_count"]] = 0
    
    for method in ["sobol","ehvi","parego"]:
        for n_unique in stats_dict[f"{x_axis}_{method}"]:
            col = df.columns.tolist().index(f"{method}_count")
            df.iloc[:n_unique,col] = df.iloc[:n_unique,col] + 1
        
    dfs_dict[f"{x_axis}"] = df

In [4]:
def col_idx(df,col):
    return df.columns.tolist().index(col)

In [5]:
for x_axis in ["micro_joules_per_pixel","model_bits","params"]:
    for method in ["sobol","ehvi","parego"]:
        mask = (dfs_dict[x_axis][f"{method}_count"] >= 2).to_list()

        if not all(mask):
            max_idx = mask.index(False)
            dfs_dict[x_axis].iloc[max_idx:,col_idx(dfs_dict[x_axis],method)] = None



In [6]:
for x_axis in ["micro_joules_per_pixel","model_bits","params"]:
    dfs_dict[x_axis].drop([f"{method}_count" for method in ["sobol","ehvi","parego"]],axis=1).to_csv(
        f"{x_axis}_data_bits_over_data_samples_ax_methods_avgs_adjusted_support2.csv"
    )